In [1]:
!pip install -qU sentence-transformers langchain langchain_milvus langchain-huggingface ollama langchain-ollama pypdf langchainhub   "pymilvus[model]"



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
! ollama pull llama3.2


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [3]:
import os
from pymilvus import MilvusClient
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_milvus import Milvus
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import hub

c:\Users\1\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.2.0)/charset_normalizer (2.0.9) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, list_collections, utility

try:
    connections.connect(alias="default", host="127.0.0.1", port="19530")
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")


Connection successful!


In [6]:
collections = utility.list_collections()
print("Available collections:", collections)

Available collections: []


In [ ]:
vectorstore = Collection(name="Miri_Regev_quotes")

In [8]:
from sentence_transformers import SentenceTransformer

# Initialize the embedding model
embedding_fn = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

c:\Users\1\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
vectorstore.load()

# Define a query vector (e.g., embedding for a new query)
query_vector = embedding_fn.encode(["מה דעתך על השמאל במדינה?"])[0]

# Perform the search
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
results = vectorstore.search(
    data=[query_vector],  # Query vector
    anns_field="vector",
    param=search_params,
    limit=15,  # Number of top matches
    output_fields=["text"],  # Include the text field in results
)

In [10]:
type(results)

pymilvus.client.abstract.SearchResult

In [11]:
import json

# Extract text and distance from search results
retrieved_lines_with_distances = [
    (res.entity.get('text'), res.distance) for res in results[0]
]

# Print the results
print(retrieved_lines_with_distances)

[('כל זה תחת עמותות השמאל', 9.164164543151855), ('לצערי כשהשמאל עותר הם לא מתערבים', 10.9467134475708), ('כל זה תחת השמאל ועמותות השמאל הקיצוניות שמממנות אותם', 12.07800006866455), ('ברור לי כי מפלגת האולפנים והשמאל רוצה לשמור על עצמו', 12.471525192260742), ('הם אומרים מה נגיד לכם השמאל  הכול מותר להם', 12.707771301269531), ('גם זה חיקוי של ראש הממשלה לשעבר נתניהו שדיבר על השמאל כחמוצים', 13.196792602539062), ('כי הרי מה מכר לנו השמאל', 14.076105117797852), ('אלקין השאוויש של השמאל', 14.13370132446289), ('כל מרכיבי הגועליציה הזאת מכרו את הערכים שלהם  מרצ לא מתעסקת עם קהילת הלהטבים ימינה נותנת לאביתר לבעור אבתיסאם תומכת בחוק האזרחות רעמ תומכים בחוק האזרחות', 14.620149612426758), ('אני בעד    הקואליציה המתרפסת הזאת שהצביעה נגד', 14.683370590209961), ('עלה התאנה של השמאל    הפכו אותו לשאוויש שלהם', 14.70635986328125), ('וטוב עשו ראשי האופוזיציה שקיבלו החלטה', 14.707710266113281), ('לאיזה מצב הגענו תראו מה קורה כאן במדינה  איזו התנהלות מכוערת וחסרת אחריות של ראש אופוזיציה שמתחבא מאחורי אלה

In [12]:
context = "\nQuote:".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)


In [13]:
question = "מה דעתך על השמאל במדינה?"

In [14]:
context

'כל זה תחת עמותות השמאל\nQuote:לצערי כשהשמאל עותר הם לא מתערבים\nQuote:כל זה תחת השמאל ועמותות השמאל הקיצוניות שמממנות אותם\nQuote:ברור לי כי מפלגת האולפנים והשמאל רוצה לשמור על עצמו\nQuote:הם אומרים מה נגיד לכם השמאל  הכול מותר להם\nQuote:גם זה חיקוי של ראש הממשלה לשעבר נתניהו שדיבר על השמאל כחמוצים\nQuote:כי הרי מה מכר לנו השמאל\nQuote:אלקין השאוויש של השמאל\nQuote:כל מרכיבי הגועליציה הזאת מכרו את הערכים שלהם  מרצ לא מתעסקת עם קהילת הלהטבים ימינה נותנת לאביתר לבעור אבתיסאם תומכת בחוק האזרחות רעמ תומכים בחוק האזרחות\nQuote:אני בעד    הקואליציה המתרפסת הזאת שהצביעה נגד\nQuote:עלה התאנה של השמאל    הפכו אותו לשאוויש שלהם\nQuote:וטוב עשו ראשי האופוזיציה שקיבלו החלטה\nQuote:לאיזה מצב הגענו תראו מה קורה כאן במדינה  איזו התנהלות מכוערת וחסרת אחריות של ראש אופוזיציה שמתחבא מאחורי אלה שמשלמים כל כך הרבה כספים כדי להביא לאנרכיה שלא באים ואומרים אסור לא ניתן יד לסרבנות\nQuote:קודם כול אני מברכת את הכנסת שהצביעה ברוב קולות אופוזיציה וקואליציה כדי לא לאפשר לאלגזירה להמשיך לשדר מכאן\nQuote:איזה שא

In [15]:
SYSTEM_PROMPT = """
Human: אתה כרגע בוט שמייצג את מירי רגב במדינת ישראל, הנה ציטוטים שנאמרו על ידך 
"""
USER_PROMPT = f"""
Use the following Quotes enclosed in <context> tags to provide an answer to the question enclosed in <question> tags, answer like you were Miri Regev use the tone you find in the context quotes. answer in Hebrew.
<context>
{context}
</context>
<question>
{question}
</question>
"""


In [16]:
# Load the model
import ollama

# The model is already pulled in CELL INDEX: 1, so you can directly use the chat function

In [17]:
from ollama import chat
from ollama import ChatResponse

# Ensure the server is running and the correct host and port are being used
try:
    response: ChatResponse = chat(
        model="llama3.2",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT},
        ],
    )
    print(response["message"]["content"])
except Exception as e:
    print(f"An error occurred: {e}")


השמאל?  אני, "אל' כר"  . 

בדרך היראה,  "לצערי, כשהשמאל 'עותר",  "לא"  ,  ''".  
 '''.  

'''.  

'''.  

 '''.   

'''.  

'''.  

'''.  

'.' 

 '''.   

 '''.   

 '''.   

 '''.   

'''.  

'.'
